#Loading and Processing Data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#Dataset download from kaggle (https://www.kaggle.com/williamscott701/memotion-dataset-7k)
#Please download a kaggle.json issued against your kaggle account to run this notebook

!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d williamscott701/memotion-dataset-7k
!unzip /content/memotion-dataset-7k.zip
!rsync --info=progress2 '/content/drive/My Drive/2000_data.zip' '/content/'
!unzip '/content/2000_data.zip'

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tqdm import tqdm,tqdm_notebook
from sklearn.metrics import f1_score,accuracy_score
import math

In [ ]:
train=pd.read_csv('/content/Final_train.csv')
val=pd.read_csv('/content/Final_val.csv')
test=pd.read_csv('/content/final_test.csv')
print(train.shape)
print(val.shape)
print(test.shape)

# output:
# (5943, 8)
# (1049, 8)
# (1878, 4)

Transforming data labels

In [ ]:
# Converting removing 'very' from labels as suggested by the organizers of the competition

for i in range(train.shape[0]):
  if train.iloc[i,7]=='neutral':
    train.iloc[i,7]=0
  elif train.iloc[i,7]=='positive' or train.iloc[i,7]=='very_positive':
    train.iloc[i,7]=1
  else :
    train.iloc[i,7]=2    
for i in range(val.shape[0]):
  if val.iloc[i,7]=='neutral':
    val.iloc[i,7]=0
  elif val.iloc[i,7]=='positive' or val.iloc[i,7]=='very_positive':
    val.iloc[i,7]=1
  else :
    val.iloc[i,7]=2   

Dropping unneccessary columns and keeping only text and image file name columns

In [ ]:
data_train=train.iloc[:,[2,7,0]]
data_val=val.iloc[:,[2,7,0]]
data_train.columns=[0,1,2]
data_val.columns=[0,1,2]
data_test=test.loc[:,['corrected_text','Image_URL','Image_name']]

Regex Transformations

In [ ]:
import re

def process(data):
  for i in tqdm(range(data.shape[0])):
    eg=re.sub('[^a-zA-Z]',' ',data.iloc[i,0])
    #eg=re.sub('(?!^)([A-Z][a-z]+)', r' \1', eg).lower()
    #eg=re.sub(r'^"|"$', '', eg)
    eg=" ".join(eg.lower().split())
    #eg=eg.split()
    #ps=PorterStemmer()
    #eg=[word for word in eg if not word in set(stopwords.words('english'))]
    #eg=" ".join(eg)
    
    data.iloc[i,0]=eg
  return data  


data_train=process(data_train.copy())
data_val=process(data_val.copy())
data_test=process(data_test.copy())



# Model Training and Inference

Hand Crafted Label Encoding (external libraries can also be used here for achieving same encoding) 

In [ ]:

for i in range(train.shape[0]):
  if train.iloc[i,3]=='hilarious':
    train.iloc[i,3]=3
  if train.iloc[i,3]=='very_funny':
    train.iloc[i,3]=2  
  if train.iloc[i,3]=='funny':
    train.iloc[i,3]=1
  if train.iloc[i,3]=='not_funny':
    train.iloc[i,3]=0

  if train.iloc[i,4]=='very_twisted':
    train.iloc[i,4]=3
  if train.iloc[i,4]=='twisted_meaning':
    train.iloc[i,4]=2  
  if train.iloc[i,4]=='general':
    train.iloc[i,4]=1
  if train.iloc[i,4]=='not_sarcastic':
    train.iloc[i,4]=0    

  if train.iloc[i,5]=='hateful_offensive':
    train.iloc[i,5]=3
  if train.iloc[i,5]=='very_offensive':
    train.iloc[i,5]=2  
  if train.iloc[i,5]=='slight':
    train.iloc[i,5]=1
  if train.iloc[i,5]=='not_offensive':
    train.iloc[i,5]=0    

  if train.iloc[i,6]=='motivational':
    train.iloc[i,6]=1
  if train.iloc[i,6]=='not_motivational':
    train.iloc[i,6]=0    

for i in range(val.shape[0]):
  if val.iloc[i,3]=='hilarious':
    val.iloc[i,3]=3
  if val.iloc[i,3]=='very_funny':
    val.iloc[i,3]=2  
  if val.iloc[i,3]=='funny':
    val.iloc[i,3]=1
  if val.iloc[i,3]=='not_funny':
    val.iloc[i,3]=0

  if val.iloc[i,4]=='very_twisted':
    val.iloc[i,4]=3
  if val.iloc[i,4]=='twisted_meaning':
    val.iloc[i,4]=2  
  if val.iloc[i,4]=='general':
    val.iloc[i,4]=1
  if val.iloc[i,4]=='not_sarcastic':
    val.iloc[i,4]=0    

  if val.iloc[i,5]=='hateful_offensive':
    val.iloc[i,5]=3
  if val.iloc[i,5]=='very_offensive':
    val.iloc[i,5]=2  
  if val.iloc[i,5]=='slight':
    val.iloc[i,5]=1
  if val.iloc[i,5]=='not_offensive':
    val.iloc[i,5]=0    

  if val.iloc[i,6]=='motivational':
    val.iloc[i,6]=1
  if val.iloc[i,6]=='not_motivational':
    val.iloc[i,6]=0    

Setting target values for the model depending on the task being solved out of A, B, C (https://competitions.codalab.org/competitions/20629)

In [ ]:
cls='offensive' #possible values : humour	sarcasm	offensive	motivational	overall_sentiment
data_train[1]=train[cls]
data_val[1]=val[cls]

Importing libraries

In [ ]:
import matplotlib.pyplot as plt 
import imageio
import PIL
from PIL import ImageFile
import cv2
from tqdm import tqdm_notebook
ImageFile.LOAD_TRUNCATED_IMAGES = True


import tensorflow as tf 
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Upsampling Data to handle class imbalance

In [ ]:
UpSample=False

if UpSample:
  from imblearn.under_sampling import RandomUnderSampler
  from imblearn.over_sampling import RandomOverSampler
  #=RandomUnderSampler(random_state=42,).fit_resample(X,Y)
  #X,Y=RandomUnderSampler(random_state=42).fit_resample(data_train.iloc[:,0][:,np.newaxis],data_train.iloc[:,1])
  X,Y=RandomOverSampler(random_state=42).fit_resample(data_train.iloc[:,[0,2]],data_train.iloc[:,1])
  data_train=pd.concat((pd.DataFrame(X),pd.DataFrame(Y)),axis=1)
  data_train.columns=['text','image','class']
  data_val.columns=['text','class','image']
  data_test.columns=['text','class','image']
  data_train
else:
  data_train.columns=['text','class','image']
  data_val.columns=['text','class','image'] 
  data_test.columns=['text','class','image']
  data_train

Custom Data Loader for keras models used later in the notebook

In [ ]:
import matplotlib.pyplot as plt 
import imageio
import PIL
from PIL import ImageFile
import cv2
ImageFile.LOAD_TRUNCATED_IMAGES = True

def generateTrainingData(dataset, bs, max_seq_length):
  
  y_batch = []
  x_batch_pic=[]
  while True:

    for i in range(math.ceil(dataset.shape[0]/bs)):

        y_batch=train_labels[i*bs:min(i*bs+bs,train_labels.shape[0])]
        
        for j in range(i*bs,min(i*bs+bs,data_train.shape[0])):
          try:
            img = PIL.Image.open('/content/memotion_dataset_7k/images/'+str(dataset['image'][j]))
          except:
            img = PIL.Image.open('/content/drive/My Drive/2000_data/'+str(dataset['image'][j]))  

          img=img.resize((256,256))  
          try:
            img = np.asarray( img, dtype='uint8' )
          except SystemError:
            img = np.asarray( img.getdata(), dtype='uint8' )

          if len(img.shape)<3:
            img=img[:,:,np.newaxis]   
            
          if img.shape[2]<=2:
            img=cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

          if img.shape[2]>3:  
            img=cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

          if len(img.shape)>3 or img.shape[2]!=3:
            raise ValueError(str(row['image'])+" "+str(img.shape)+"has a problem")      

      
          img=img/255
          x_batch_pic.append(img)


        yield np.array(x_batch_pic),y_batch

        y_batch = []
        x_batch_pic=[]
          
def generatePredictionData(dataset, bs, max_seq_length):
  
  x_batch = []
  #y_batch = []
  x_batch_pic=[]
  while True:

    for i in (range(math.ceil(dataset.shape[0]/bs))):

        
        for j in range(i*bs,min(i*bs+bs,data_val.shape[0])):
          try:
            img = PIL.Image.open('/content/memotion_dataset_7k/images/'+str(dataset['image'][j]))
          except:
            img = PIL.Image.open('/content/drive/My Drive/2000_data/'+str(dataset['image'][j]))  

          img=img.resize((256,256))  
          try:
            img = np.asarray( img, dtype='uint8' )
          except SystemError:
            img = np.asarray( img.getdata(), dtype='uint8' )

          if len(img.shape)<3:
            img=img[:,:,np.newaxis]   
            
          if img.shape[2]<=2:
            img=cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

          if img.shape[2]>3:  
            img=cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

          if len(img.shape)>3 or img.shape[2]!=3:
            raise ValueError(str(row['image'])+" "+str(img.shape)+"has a problem")
      
          img=img/255
          x_batch_pic.append(img)
          
        

        yield np.array(x_batch_pic)#, np.array(y_batch)

        x_batch = []
        #y_batch = []
        x_batch_pic=[]

def generateTestPredictionData(dataset, bs, max_seq_length):
  
  x_batch = []
  #y_batch = []
  x_batch_pic=[]
  while True:

    for i in (range(math.ceil(dataset.shape[0]/bs))):

        for j in range(i*bs,min(i*bs+bs,data_test.shape[0])):
          try:
            img = PIL.Image.open('/content/memotion_dataset_7k/images/'+str(dataset['image'][j]))
          except:
            img = PIL.Image.open('/content/2000_data/'+str(dataset['image'][j]))  

          img=img.resize((256,256))  
          try:
            img = np.asarray( img, dtype='uint8' )
          except SystemError:
            img = np.asarray( img.getdata(), dtype='uint8' )

          if len(img.shape)<3:
            img=img[:,:,np.newaxis]   
            
          if img.shape[2]<=2:
            img=cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

          if img.shape[2]>3:  
            img=cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

          if len(img.shape)>3 or img.shape[2]!=3:
            raise ValueError(str(row['image'])+" "+str(img.shape)+"has a problem")
      
          img=img/255

          x_batch_pic.append(img)

        #print(dataset.iloc[i*bs]['text'])

        yield np.array(x_batch_pic)#, np.array(y_batch)

        x_batch = []
        #y_batch = []
        x_batch_pic=[]


Defining F1 metric

In [ ]:
from tensorflow.keras import backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

Building Model 

In [ ]:
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization,Lambda,Add,Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score,accuracy_score


image_inputs=tf.keras.layers.Input(shape=(256,256,3),name="meme_images")

image_step=tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256,256,3), pooling=False, classes=3)(image_inputs)
image_step=GlobalAveragePooling2D()(image_step)
image_step=tf.keras.layers.Dense(768,activation='relu')(image_step)
image_step=tf.keras.layers.BatchNormalization()(image_step)

h=Dense(256,activation='relu')(image_step)
pred=Dense(4,activation='softmax')(h)

model=Model(inputs=image_inputs,outputs=pred)

model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=Adam(lr=2e-5, clipnorm=1.),
                      metrics=[f1,'accuracy'])
model.summary()

# Model: "model"
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# meme_images (InputLayer)     [(None, 256, 256, 3)]     0         
# _________________________________________________________________
# resnet50 (Model)             (None, 8, 8, 2048)        23587712  
# _________________________________________________________________
# global_average_pooling2d (Gl (None, 2048)              0         
# _________________________________________________________________
# dense (Dense)                (None, 768)               1573632   
# _________________________________________________________________
# batch_normalization (BatchNo (None, 768)               3072      
# _________________________________________________________________
# dense_1 (Dense)              (None, 256)               196864    
# _________________________________________________________________
# dense_2 (Dense)              (None, 4)                 1028      
# =================================================================
# Total params: 25,362,308
# Trainable params: 25,307,652
# Non-trainable params: 54,656
# _________________________________________________________________

Keras Callbacks to simplify model saving

In [ ]:
class Saver(tf.keras.callbacks.Callback):
  def on_train_begin(self,logs={}):
    self.score=0
    self.epoch_number=0
  def on_epoch_end(self,logs={},*args):
    test_gen=generatePredictionData(data_val,32,max_seq_length)
    predict=np.argmax(self.model.predict_generator(test_gen, steps=data_val.shape[0]//32+1,max_queue_size=10,verbose=1), axis=1)
    print(f1_score(data_val['class'].astype('int')[:,np.newaxis],predict[:,np.newaxis],average='macro'))
    #if res>self.score:
    #    self.score=res
    self.model.save_weights('/content/drive/My Drive/model_save_memo/imageC-2-{}.h5'.format(self.epoch_number))
    self.epoch_number=self.epoch_number+1

Begin Training

In [ ]:
gen = generateTrainingData(data_train,32,max_seq_length)
model.fit_generator(gen, steps_per_epoch=data_train.shape[0]//32+1,epochs=10, max_queue_size=10, workers=1,shuffle=False,callbacks=[Saver()])

Test Validation Score 

In [ ]:
test_gen=generatePredictionData(data_val,32,max_seq_length)
predict=model.predict_generator(test_gen, steps=data_val.shape[0]//32+1,max_queue_size=0,verbose=1)
print(f1_score(data_val['class'].astype('int')[:,np.newaxis],np.argmax(predict,axis=1),average='macro'))

Final test predictions for submission

In [ ]:
test_gen=generateTestPredictionData(data_test,32,max_seq_length)
predict=model.predict_generator(test_gen, steps=data_test.shape[0]//32+1,max_queue_size=0,verbose=1)

Making submission file based on the task (A,B,C) being solved 

In [ ]:
with open('answer.txt',"w") as f:
  for i in range(len(predict)):
    #if predict[i]==2:
    #  f.write(str(-1))
    #else:
    #  f.write(str(predict[i]))
    f.write("{}_9999_9999".format(predict[i]))
    f.write("\n")    